# Integrantes
- Javier Chen 22153
- Gustavo Cruz 22779

In [29]:
import numpy as np
import altair as alt
import pandas as pd

In [30]:
# Inicializa los pesos y sesgos aleatoriamente para una red
def init_parameters(n_features, n_neurons, n_output):
    np.random.seed(100)  # Para reproducibilidad
    W1 = np.random.uniform(size=(n_features, n_neurons))  # Pesos capa oculta
    b1 = np.random.uniform(size=(1, n_neurons))           # Sesgos capa oculta

    W2 = np.random.uniform(size=(n_neurons, n_output))    # Pesos capa de salida
    b2 = np.random.uniform(size=(1, n_output))            # Sesgos capa de salida

    return {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2
    }

In [31]:
def linear_function(W, X, b): # Función de propagación lineal: Z = XW + b
    return (X @ W)+ b

In [32]:
def sigmoid_func(Z): # Introduce no linealidad al modelo y comprime los valores a [0,1]
    return 1 / (1 + np.exp(-Z))

In [33]:
def cost_function(A, y): # Evalúa qué tan buena es la predicción con respecto al valor real
    epsilon = 1e-15
    A = np.clip(A, epsilon, 1 - epsilon)
    return -np.mean(y * np.log(A) + (1 - y) * np.log(1 - A))

In [34]:
# Función de predicción: realiza una pasada hacia adelante y aplica umbral de 0.5
# Retorna 0 o 1 según la salida final
def predict(X, W1, W2, b1, b2):
    Z1 = linear_function(W1, X, b1)
    S1 = sigmoid_func(Z1)
    Z2 = linear_function(W2, S1, b2)
    S2 = sigmoid_func(Z2)
    return np.where(S2 >= 0.5, 1, 0)

In [35]:
# Ejecuta el ciclo de entrenamiento por un número de iteraciones usando backpropagation
def fit(X, y, n_features=2, n_neurons=3, n_output=1, iterations=10, eta=0.001):

  # Inicialización de parámetros
    params = init_parameters(
        n_features=n_features,
        n_neurons=n_neurons,
        n_output=n_output
    )

    errors = []

   # Ciclo de entrenamiento (forward + backward)
    for _ in range(iterations):

        # FORWARD PROPAGATION
        Z1 = linear_function(params['W1'], X, params['b1'])
        S1 = sigmoid_func(Z1)
        Z2 = linear_function(params['W2'], S1, params['b2'])
        S2 = sigmoid_func(Z2)

        # Cálculo del error
        error = cost_function(S2, y)
        errors.append(error)

        # BACKPROPAGATION
        # Capa de salida: error directo
        delta2 = S2 - y

        W2_gradients = S1.T @ delta2
        params["W2"] = params["W2"] - W2_gradients * eta
        params["b2"] = params["b2"] - np.sum(delta2, axis=0, keepdims=True) * eta


        # Capa oculta: propagación del error con derivada de sigmoide
        delta1 = (delta2 @ params["W2"].T) * S1 * (1 - S1)
        W1_gradients = X.T @ delta1
        params["W1"] = params["W1"] - W1_gradients * eta
        params["b1"] = params["b1"] - np.sum(delta1, axis=0, keepdims=True) * eta

    return errors, params

In [36]:
# Datos XOR para entrenamiento
y = np.array([[0, 1, 1, 0]]).T
X = np.array([[0, 0, 1, 1]
              ,[0, 1, 0, 1]]).T

In [37]:
errors, params = fit(X, y, iterations=5000, eta = 0.1) # Entrenamiento del modelo

In [38]:
y_pred = predict(X, params["W1"], params["W2"], params["b1"], params["b2"]) # Predicción sobre los mismos datos
# Evaluación de la exactitud (accuracy)
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100
print('Multi-layer perceptron accuracy: %.2f%%' % accuracy)


Multi-layer perceptron accuracy: 100.00%


In [39]:
# Gráfico del error por iteración
alt.data_transformers.disable_max_rows()
df = pd.DataFrame({"errors":errors, "time-step": np.arange(0, len(errors))})
alt.Chart(df).mark_line().encode(x="time-step", y="errors").properties(title='Chart 2')


alt.Chart(...)

# Mismo Dataset en Framework Pytorch

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import altair as alt

In [41]:
# Datos de entrada: problema clásico de clasificación XOR
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T  # Shape (4, 2)
y = np.array([[0, 1, 1, 0]]).T  # Shape (4, 1)

# Conversión de los datos a tensores de PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Definición del modelo neuronal (red simple de 2 capas totalmente conectadas)
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)  # Capa oculta: 2 neuronas de entrada → 16 neuronas
        self.fc2 = nn.Linear(16, 1)  # Capa de salida: 16 → 1 neurona de salida
        self.sigmoid = nn.Sigmoid()    # Función de activación no lineal

     # Método forward: define el flujo hacia adelante del modelo (predicción)
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))  # Activación de la capa oculta
        x = self.sigmoid(self.fc2(x))  # Activación de la capa de salida
        return x

# Instanciación del modelo
model = SimpleNet()

# Definición de la función de pérdida
criterion = nn.BCELoss()

# Optimizador: actualiza los pesos de forma eficiente durante el entrenamiento
optimizer = optim.Adam(model.parameters())

# Entrenamiento del modelo
errors = []   # Lista para guardar el error en cada época
epochs = 3000 # Número de épocas

for epoch in range(epochs):
    optimizer.zero_grad()              # Reinicia los gradientes acumulados
    outputs = model(X_tensor)          # Forward pass: calcula la salida actual de la red
    loss = criterion(outputs, y_tensor)  # Calcula el error entre predicción y etiqueta real
    loss.backward()                    # Backward pass: calcula gradientes automáticamente
    optimizer.step()                   # Actualiza los pesos con los gradientes
    errors.append(loss.item())         # Guarda el error actual para graficarlo

# Visualización del error a lo largo del tiempo (entrenamiento)
df2 = pd.DataFrame({"errors": errors, "time-step": np.arange(epochs)})

alt.Chart(df2).mark_line().encode(
    x="time-step",
    y="errors"
).properties(title='Pytorch')

alt.Chart(...)

In [42]:
df_numpy = pd.DataFrame({
    "errors": errors,
    "time-step": np.linspace(0, 1, len(errors)),
    "implementation": "NumPy"
})

df_pytorch = pd.DataFrame({
    "errors": df2["errors"],
    "time-step": np.linspace(0, 1, len(df2["errors"])),
    "implementation": "PyTorch"
})

df_total = pd.concat([df_numpy, df_pytorch], ignore_index=True)

alt.Chart(df_total).mark_line().encode(
    x=alt.X("time-step:Q", title="Progreso del entrenamiento (0 a 1)"),
    y=alt.Y("errors:Q", title="Error"),
    color="implementation:N"
).properties(title="Comparación normalizada del error")



alt.Chart(...)

## Preguntas

1. ¿Existen cambios de arquitectura en las 2 redes implementadas?
  - Sí, para el modelo en numpy usamos 2 neuronas de entrada, 3 ocultas y una de salida, y en pytorch usamos 2 de entrada, 16 ocultas y una salida.
2. ¿Exiten diferencias en la velocidad de convergencia entre las 2 redes?
  - Sí, la red de pytorch es más rápida, y puede deberse a la diferencia de arquitectura anteriormente mencionada, además que implementa un optimizador de Adam y se ve en la gráfica de pytorch que la curva es más suave que la de numpy y alcanza un nivel de error cercano a 0 con menos pasos de tiempo que numpy.
3. En la función init_parameters, ¿qué sucede si inicializamos los pesos en 0 en
lugar de valores aleatorios? ¿y los bias?
  - Todos los nodos de una capa van a hacer el mismo cálculo de propagación y van a recibir el mismo gradiente y no van a aprender nada útil. Por el contrario, el bias inicializado en 0, no afectará al aprendizaje, por que no están conectados entre sí, y cada neurona al final aprenderá algo diferente. Lo único es que al principio las neuronas no van a tener su "opinion" pero con cada iteración se va a ir corrigiendo.